In [33]:
import numpy as np


class KalmanFilter:
    def __init__(
        self, dt: float, u_x: float, u_y: float, x_std_meas: float, y_std_meas: float
    ):
        self.dt = dt
        self.u_x = u_x
        self.u_y = u_y
        self.std_acc_x = y_std_meas
        self.std_acc_y = x_std_meas
        self.time_state = 0
        self.x = np.array([[0, 0, 0, 0]]).T
        self.A = np.array(
            [[1, 0, self.dt, 0], [0, 1, 0, self.dt], [0, 0, 1, 0], [0, 0, 0, 1]]
        )
        self.B = np.array(
            [
                [(self.dt**2) / 2, 0],
                [0, (self.dt**2) / 2],
                [self.dt, 0],
                [0, self.dt],
            ]
        )
        self.H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])
        self.Q = (
            np.array(
                [
                    [(self.dt**4) / 4, 0, (self.dt**3) / 2, 0],
                    [0, (self.dt**4) / 4, 0, (self.dt**3) / 2],
                    [(self.dt**3) / 2, 0, self.dt**2, 0],
                    [0, (self.dt**3) / 2, 0, self.dt**2],
                ]
            )
            * self.std_acc_x**2
        )
        self.R = np.array([[self.std_acc_x**2, 0], [0, self.std_acc_y**2]])
        self.P = np.eye(self.A.shape[1])

    def predict(self):
        self.x = np.dot(self.A, self.x) + np.dot(
            self.B, np.array([[self.u_x, self.u_y]]).T
        )
        self.P = np.dot(np.dot(self.A, self.P), self.A.T) + self.Q
        self.time_state += self.dt
        return self.x

    def update(self, z):
        S = np.dot(np.dot(self.H, self.P), self.H.T) + self.R
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))
        self.x = self.x + np.dot(K, (z - np.dot(self.H, self.x)))
        self.P = self.P - np.dot(np.dot(K, self.H), self.P)

In [34]:
import cv2
import numpy as np


def detect(frame):
    # Convert frame from BGR to GRAY
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # cv2.imshow('gray', gray)

    # Edge detection using Canny function
    img_edges = cv2.Canny(gray, 50, 190, 3)
    # cv2.imshow('img_edges', img_edges)

    # Convert to black and white image
    ret, img_thresh = cv2.threshold(img_edges, 254, 255, cv2.THRESH_BINARY)
    # cv2.imshow('img_thresh', img_thresh)

    # Find contours
    contours, _ = cv2.findContours(
        img_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    # Set the accepted minimum & maximum radius of a detected object
    min_radius_thresh = 3
    max_radius_thresh = 30

    centers = []
    for c in contours:
        # ref: https://docs.opencv.org/trunk/dd/d49/tutorial_py_contour_features.html
        (x, y), radius = cv2.minEnclosingCircle(c)
        radius = int(radius)

        # Take only the valid circle(s)
        if (radius > min_radius_thresh) and (radius < max_radius_thresh):
            centers.append(np.array([[x], [y]]))
    # cv2.imshow("contours", img_thresh)
    return centers

In [35]:
import matplotlib.pyplot as plt
from PIL import Image

kf = KalmanFilter(dt=0.1, u_x=1, u_y=1, x_std_meas=0.1, y_std_meas=0.1)
cap = cv2.VideoCapture("randomball.avi")
fig = plt.figure()
frames = []
for i in range(125):
    frame = cap.read()[1]

    detected_point = detect(frame)[0]
    detected_point = detected_point.reshape(-1)
    kf.predict()
    kf.update(detected_point)
    print(kf.x)
    # draw a green square to visualize the current prediction
    cv2.rectangle(
        frame,
        (int(kf.x[0, 0] - 10), int(kf.x[0, 1] - 10)),
        (int(kf.x[0, 0] + 10), int(kf.x[0, 1] + 10)),
        (0, 255, 0),
        2,
    )
    frames.append(frame)

images = [Image.fromarray(f, mode="RGB") for f in frames]
output_file = "randomball.gif"
images[0].save(
    output_file,
    format="GIF",
    append_images=images[1:],
    save_all=True,
    duration=10,
    loop=0,
)

[[278.95685117 155.46083371]
 [278.95685117 155.46083371]
 [ 27.72036928  15.49243248]
 [ 27.72036928  15.49243248]]
[[281.72379582 157.00502544]
 [281.72379582 157.00502544]
 [ 27.76990352  15.54217053]
 [ 27.76990352  15.54217053]]
[[282.65073818 157.52308453]
 [282.65073818 157.52308453]
 [ 18.6395368   10.46156612]
 [ 18.6395368   10.46156612]]
[[273.59361289 159.70186325]
 [273.59361289 159.70186325]
 [-24.79854848  15.05489879]
 [-24.79854848  15.05489879]]
[[268.80191699 160.74547104]
 [268.80191699 160.74547104]
 [-32.15189966  13.6529243 ]
 [-32.15189966  13.6529243 ]]
[[266.30102899 161.24270104]
 [266.30102899 161.24270104]
 [-30.15535031  11.41850813]
 [-30.15535031  11.41850813]]
[[264.96952916 161.48103342]
 [264.96952916 161.48103342]
 [-26.22579727   9.44635411]
 [-26.22579727   9.44635411]]
[[261.38869567 164.86821437]
 [261.38869567 164.86821437]
 [-28.03938672  14.38875555]
 [-28.03938672  14.38875555]]
[[259.11622525 167.0849558 ]
 [259.11622525 167.0849558 ]
 [-27.

<Figure size 640x480 with 0 Axes>